# W3P2 - PART3 - yelp_foursquareEDA

assignment file part 2

In [110]:
from datetime import datetime
import pandas as pd
import requests
import json
import os
import warnings
warnings.filterwarnings('ignore')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. Assuming the average comfortable walking speed below, the radius of 1km should list sites available within a 15min walking distance from each bike station.

$$
1km * hour/4km 
$$

In [2]:
# Import data from file
df = pd.read_csv('../data/df_ctybks_toronto.csv')
df.shape

(655, 19)

Create lat and long series from the ctyBks table:

In [3]:
lat = df['latitude']
long = df['longitude']
print(f'{lat[5]},{long[5]}')

43.657763,-79.389165


In [4]:
numStns = len(lat)

In [5]:
def fsqStationPOI(lat, long):
  '''
  input:
  output:
  '''
  api_key = os.environ["FOURSQUARE_API_KEY"]
  fields = 'fsq_id,name,geocodes,categories,distance,rating,hours,hours_popular,popularity,features'

  headers = {
      "accept": "application/json",
      "Authorization": api_key
  }

  fsq_url = (f'https://api.foursquare.com/v3/places/search?ll={lat},{long}&fields={fields}&radius=1000&&limit=50')
  
  fsqStnPOI_json = requests.get(fsq_url, headers=headers).json()

  list_of_dict = []
  for poi in fsqStnPOI_json['results']:
    categories = poi.get('categories', None)
    cat_id = categories[0]['id'] if len(categories) > 0 else None
    category_name = categories[0]['name'] if len(categories) > 0 else None

    poi_dict = {
            'lat-long': f'{lat},{long}'
          , 'fsq_id': poi['fsq_id']
          , 'cat_id': cat_id
          , 'category_name': category_name
          , 'categories': categories
          , 'name': poi.get('name', None)
          , 'distance': poi.get('distance', None)
          , 'latitude': poi.get('geocodes', {}).get('main', {}).get('latitude', None)
          , 'longitude': poi.get('geocodes', {}).get('main', {}).get('longitude', None)
          , 'address': poi.get('location', {}).get('formatted_address', None)
          , 'popularity': poi.get('popularity', None)
          , 'open_now': poi.get('hours', {}).get('open_now', None)
          , 'rating': poi.get('rating', None)
          , 'rating_count': poi.get('stats', {}).get('total_ratings', None)
          , 'my_timestamp': datetime.now()
      }
      
    list_of_dict.append(poi_dict)
  return pd.DataFrame(list_of_dict)


In [8]:
print(f'{lat[5]},{long[5]}')

43.657763,-79.389165


In [9]:
# Test
fsqStnPOI_df = fsqStationPOI(lat[5], long[5])
fsqStnPOI_df.head(1)

,lat-long,fsq_id,cat_id,category_name,categories,name,distance,latitude,longitude,address,popularity,open_now,rating,rating_count,my_timestamp
0,"43.657763,-79.389165",537d4d6d498ec171ba22e7fe,13034,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Jimmy's Coffee,294,43.658525,-79.385436,None,0.972089,False,8.5,None,2022-12-03 19:38:44.660853


Generate a list of DataFrames now for each station site:

In [137]:
fsq_dfs_list = []
for i in range(numStns):
  poi_df = fsqStationPOI(lat[i], long[i])
  fsq_dfs_list.append(poi_df)

Concatenate the DataFrames to create one large one.

In [226]:
df_fsq = pd.DataFrame()
x = pd.DataFrame()

# concatenate full df object of all stn site data:
for poi_df in fsq_dfs_list:
    x = pd.concat([df_fsq, poi_df])
    df_fsq = x

In [227]:
df_fsq.shape

(32709, 15)

## Cleaning and Normalizing the nested table values:

The function which created the DataFrame made sure that each POI had at least one category name and ID, but we can see above that many have two or even three which can be normalized to flatten the hierarchy.

In [228]:
# Normalize the categories:
df_fsqcat = pd.json_normalize(df_fsq['categories'][0])
df_fsqcat.head()

,0,1,2
0,"{'id': 17065, 'name': 'Farmers' Market', 'icon...","{'id': 17069, 'name': 'Grocery Store / Superma...",None
1,"{'id': 13035, 'name': 'Coffee Shop', 'icon.pre...",None,None
2,"{'id': 17119, 'name': 'Bicycle Store', 'icon.p...",None,None
3,"{'id': 10039, 'name': 'Music Venue', 'icon.pre...",None,None
4,"{'id': 13011, 'name': 'Gay Bar', 'icon.prefix'...","{'id': 13016, 'name': 'Lounge', 'icon.prefix':...","{'id': 13065, 'name': 'Restaurant', 'icon.pref..."


In [229]:
# normalize the resulting 3 columns into 3 tables in the example below:
df_fsqcat0 = pd.json_normalize(df_fsqcat[0])
df_fsqcat1 = pd.json_normalize(df_fsqcat[1])
df_fsqcat2 = pd.json_normalize(df_fsqcat[2])
df_fsqcat0.head()

,id,name,icon.prefix,icon.suffix
0,17065.0,Farmers' Market,https://ss3.4sqi.net/img/categories_v2/shops/f...,.png
1,13035.0,Coffee Shop,https://ss3.4sqi.net/img/categories_v2/food/co...,.png
2,17119.0,Bicycle Store,https://ss3.4sqi.net/img/categories_v2/shops/b...,.png
3,10039.0,Music Venue,https://ss3.4sqi.net/img/categories_v2/arts_en...,.png
4,13011.0,Gay Bar,https://ss3.4sqi.net/img/categories_v2/nightli...,.png


In [230]:
# Add new category labels back onto the df: 
df_fsq['cat1_id'] = (df_fsqcat0['id'].astype('Int64'))
df_fsq['cat1_name'] = df_fsqcat0['name']
df_fsq['cat1_icon'] = df_fsqcat0['icon.prefix']
df_fsq['cat1_icon.suffix'] = df_fsqcat0['icon.suffix']

df_fsq['cat2_id'] = df_fsqcat1['id'].astype('Int64')
df_fsq['cat2_name'] = df_fsqcat1['name']
df_fsq['cat2_icon'] = df_fsqcat1['icon.prefix']
df_fsq['cat2_icon.suffix'] = df_fsqcat1['icon.suffix']

df_fsq['cat3_id'] = df_fsqcat2['id'].astype('Int64')
df_fsq['cat3_name'] = df_fsqcat2['name']
df_fsq['cat3_icon'] = df_fsqcat2['icon.prefix']
df_fsq['cat3_icon.suffix'] = df_fsqcat2['icon.suffix']

# Remove the now redundant 'category' and 'name' columns:
df_fsq = df_fsq.drop(['categories', 'category_name', 'cat_id', ], axis = 1)

df_fsq.head(1)

,lat-long,fsq_id,name,distance,latitude,longitude,address,popularity,open_now,rating,...,cat1_icon,cat1_icon.suffix,cat2_id,cat2_name,cat2_icon,cat2_icon.suffix,cat3_id,cat3_name,cat3_icon,cat3_icon.suffix
0,"43.665269,-79.319796",4deb8ba688774880e3387c0c,Leslieville Farmers Market,40,43.664679,-79.319687,None,0.86858,False,8.6,...,https://ss3.4sqi.net/img/categories_v2/shops/f...,.png,17069,Grocery Store / Supermarket,https://ss3.4sqi.net/img/categories_v2/shops/f...,.png,<NA>,NaN,NaN,NaN


#### SAVE IT!!

In [231]:
df_fsq.to_csv(f'../data/fsqPOI/df_fsq{datetime.now()}.csv', index=False)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

### FOURSQUARE POI:

In [232]:
# Clone a working copy:
dff = df_fsq.copy()

# Drop columns from our copy df not currently needed:
dff = dff.drop(['fsq_id', 'cat1_icon', 'cat2_icon', 'cat3_icon', 'cat1_icon.suffix', 'cat2_icon.suffix', 'cat3_icon.suffix', 'address'], axis = 1)
dff.head(2)

,lat-long,name,distance,latitude,longitude,popularity,open_now,rating,rating_count,my_timestamp,cat1_id,cat1_name,cat2_id,cat2_name,cat3_id,cat3_name
0,"43.665269,-79.319796",Leslieville Farmers Market,40,43.664679,-79.319687,0.868580,False,8.6,None,2022-12-04 12:07:13.609094,17065,Farmers' Market,17069,Grocery Store / Supermarket,<NA>,NaN
1,"43.665269,-79.319796",McDonald's,62,43.664993,-79.319222,0.995501,True,7.4,None,2022-12-04 12:07:13.609111,13035,Coffee Shop,<NA>,NaN,<NA>,NaN


Convert the timestamp data to the Toronto's time zone:

In [233]:
from datetime import datetime
import pytz

In [234]:
# create both timezone objects
old_timezone = pytz.timezone("US/Mountain")
new_timezone = pytz.timezone("US/Eastern")

# Confirm current timezone
print(dff['my_timestamp'].iloc[0])

# two-step process
for i in range(dff.shape[0]):
  localized_timestamp = old_timezone.localize(dff['my_timestamp'].iloc[i])
  dff['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)
  dff['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)


# Has converted to new timezone:
print(dff['my_timestamp'].iloc[0])

2022-12-04 12:07:13.609094
2022-12-04 14:07:13.609094-05:00


Put your parsed results into a DataFrame

In [235]:
dff.head(10)

,lat-long,name,distance,latitude,longitude,popularity,open_now,rating,rating_count,my_timestamp,cat1_id,cat1_name,cat2_id,cat2_name,cat3_id,cat3_name
0,"43.665269,-79.319796",Leslieville Farmers Market,40,43.664679,-79.319687,0.868580,False,8.6,None,2022-12-04 14:07:13.609094-05:00,17065,Farmers' Market,17069,Grocery Store / Supermarket,<NA>,NaN
1,"43.665269,-79.319796",McDonald's,62,43.664993,-79.319222,0.995501,True,7.4,None,2022-12-04 14:07:13.609111-05:00,13035,Coffee Shop,<NA>,NaN,<NA>,NaN
2,"43.665269,-79.319796",Rorschach Brewing,194,43.663542,-79.319882,0.984511,True,8.3,None,2022-12-04 14:07:13.609117-05:00,17119,Bicycle Store,<NA>,NaN,<NA>,NaN
3,"43.665269,-79.319796",Velotique,186,43.666157,-79.317863,0.968785,True,8.0,None,2022-12-04 14:07:13.609123-05:00,10039,Music Venue,<NA>,NaN,<NA>,NaN
4,"43.665269,-79.319796",Chick-N-Joy,129,43.665121,-79.321347,0.035783,True,6.9,None,2022-12-04 14:07:13.609129-05:00,13011,Gay Bar,13016,Lounge,13065,Restaurant
5,"43.665269,-79.319796",Jonathan Ashbridge Park,63,43.664672,-79.319780,0.952623,False,NaN,None,2022-12-04 14:07:13.609135-05:00,13034,Café,13035,Coffee Shop,13065,Restaurant
6,"43.665269,-79.319796",O Sushi,300,43.666623,-79.316861,0.936468,True,7.4,None,2022-12-04 14:07:13.609140-05:00,17096,Miscellaneous Store,<NA>,NaN,<NA>,NaN
7,"43.665269,-79.319796",Woodbine Park,379,43.662983,-79.312827,0.992987,False,8.2,None,2022-12-04 14:07:13.609145-05:00,13026,BBQ Joint,13289,Korean Restaurant,<NA>,NaN
8,"43.665269,-79.319796",Null,377,43.666781,-79.315436,0.950868,True,8.0,None,2022-12-04 14:07:13.609150-05:00,11064,Hair Salon,<NA>,NaN,<NA>,NaN
9,"43.665269,-79.319796",Lcbo,420,43.666880,-79.314844,0.976800,True,8.3,None,2022-12-04 14:07:13.609155-05:00,13236,Italian Restaurant,13302,Mediterranean Restaurant,13338,Seafood Restaurant


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [165]:
import requests
import os

def yelpStationPOI(lat, long):
  '''
  input:
  output:
  '''
  api_key = os.environ["YELP_API_KEY"]
  yelp_url = (f'https://api.yelp.com/v3/businesses/search?latitude={lat}&longitude={long}&radius=1000&limit=50')
  payload={}
  headers = {
      "accept": "application/json",
      "Authorization": f'Bearer {api_key}'
  }

  yelpStnPOI_json = requests.get(yelp_url, headers=headers, data=payload).json()           #Perform get request
  
  list_of_dict = []
  for poi in yelpStnPOI_json['businesses']:
    poi_dict = {
          'lat-long': f'{lat},{long}'
          , 'category_name': poi['categories'][0]['alias']
          , 'categories': poi['categories']
          , 'name': poi['name']
          , 'distance': poi['distance']
          , 'latitude': poi['coordinates']['latitude']
          , 'longitude': poi['coordinates']['longitude']
          , 'address': poi['location']['display_address']
          , 'price': poi.get('price', None)
          , 'is_closed': poi.get('is_closed')
          , 'rating': poi.get('rating')
          , 'rating_count': poi.get('review_count')
          , 'my_timestamp': datetime.now()
      }
      
    list_of_dict.append(poi_dict)
  return pd.DataFrame(list_of_dict)


In [166]:
# Test
yelpStnPOI_dfTest = yelpStationPOI(lat[5], long[5])
yelpStnPOI_dfTest.head(1)

,lat-long,category_name,categories,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count,my_timestamp
0,"43.657763,-79.389165",ramen,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",Sansotei Ramen,376.064159,43.655,-79.38643,"[179 Dundas Street W, Toronto, ON M5G 1Z8, Can...",$$,False,4.0,925,2022-12-04 12:44:09.123808


In [167]:
yelp_dfs_list = []
for i in range(numStns):
  poi_df = yelpStationPOI(lat[i], long[i])
  yelp_dfs_list.append(poi_df)

In [168]:
df_yelp = x = pd.DataFrame()
x = x = pd.DataFrame()

# concatenate full df object of all stn site data:
for poi_df in yelp_dfs_list:
    x = pd.concat([df_yelp, poi_df])
    df_yelp = x

In [169]:
df_yelp.head()

,lat-long,category_name,categories,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count,my_timestamp
0,"43.665269,-79.319796",egyptian,"[{'alias': 'egyptian', 'title': 'Egyptian'}, {...",Maha's,1004.992087,43.671670,-79.328530,"[226 Greenwood Avenue, Toronto, ON M4L 2R2, Ca...",$$,False,4.0,573,2022-12-04 12:44:12.200920
1,"43.665269,-79.319796",burgers,"[{'alias': 'burgers', 'title': 'Burgers'}]",The Burger's Priest,374.238073,43.666700,-79.315585,"[1636 Queen Street E, Toronto, ON M4L 1G3, Can...",$$,False,3.5,498,2022-12-04 12:44:12.200939
2,"43.665269,-79.319796",mexican,"[{'alias': 'mexican', 'title': 'Mexican'}]",Chino Locos Original,467.122546,43.664482,-79.325501,"[4 Greenwood Avenue, Toronto, ON M4L 2P4, Canada]",$,False,4.0,190,2022-12-04 12:44:12.200944
3,"43.665269,-79.319796",halal,"[{'alias': 'halal', 'title': 'Halal'}, {'alias...",Lahore Tikka House,787.473798,43.671480,-79.324510,"[1365 Gerrard Street E, Toronto, ON M4L 1Z3, C...",$$,False,3.5,537,2022-12-04 12:44:12.200950
4,"43.665269,-79.319796",italian,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",Gio Rana's Really Really Nice Restaurant,881.124117,43.663335,-79.330419,"[1220 Queen Street East, Toronto, ON M4M 1L7, ...",$$$,False,4.0,227,2022-12-04 12:44:12.200955


In [170]:
df_yelp.shape

(30828, 13)

### SAVE IT!!!

In [171]:
df_yelp.to_csv(f'../data/yelpPOI/df_yelp{datetime.now()}.csv', index=False)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

## Cleaning and Normalizing the nested table values:

### YELP POI:

The function which created the DataFrame made sure that each POI had at least one category name and alias, but we can see above that many have two or even three which can be normalized to flatten the hierarchy.

In [173]:
# Clone a working copy:
df = df_yelp.copy()

# Normalize the categories:
dfcat = pd.json_normalize(df['categories'][0])
dfcat.head()

,0,1,2
0,"{'alias': 'egyptian', 'title': 'Egyptian'}","{'alias': 'vegan', 'title': 'Vegan'}","{'alias': 'sandwiches', 'title': 'Sandwiches'}"
1,"{'alias': 'italian', 'title': 'Italian'}",None,None
2,"{'alias': 'burgers', 'title': 'Burgers'}",None,None
3,"{'alias': 'newcanadian', 'title': 'Canadian (N...",None,None
4,"{'alias': 'japanese', 'title': 'Japanese'}","{'alias': 'pubs', 'title': 'Pubs'}",None


In [174]:
# normalize the resulting 3 columns into 3 tables in the example below:
dfcat0 = pd.json_normalize(dfcat[0])
dfcat1 = pd.json_normalize(dfcat[1])
dfcat2 = pd.json_normalize(dfcat[2])
dfcat0.head()

,alias,title
0,egyptian,Egyptian
1,italian,Italian
2,burgers,Burgers
3,newcanadian,Canadian (New)
4,japanese,Japanese


In [175]:
# Add new category labels back onto the df: 
df['cat1_alias'] = dfcat0['alias']
df['cat1_title'] = dfcat0['title']

df['cat2_alias'] = dfcat1['alias']
df['cat2_title'] = dfcat1['title']

df['cat3_alias'] = dfcat2['alias']
df['cat3_title'] = dfcat2['title']

# Remove the now redundant 'category' and 'name' columns:
df = df.drop(['categories', 'category_name'], axis = 1)

df.head(1)

,lat-long,name,distance,latitude,longitude,address,price,is_closed,rating,rating_count,my_timestamp,cat1_alias,cat1_title,cat2_alias,cat2_title,cat3_alias,cat3_title
0,"43.665269,-79.319796",Maha's,1004.992087,43.67167,-79.32853,"[226 Greenwood Avenue, Toronto, ON M4L 2R2, Ca...",$$,False,4.0,573,2022-12-04 12:44:12.200920,egyptian,Egyptian,vegan,Vegan,sandwiches,Sandwiches


the addresses look chunky which diminishes the aesthetic of the table, and I expect this format might be useful for printing them in mailing format.  I won't be using it for my analysis at this point anyway, but I would certainly research this more before I made decisions about changing the format.  


In [176]:
# Drop columns from our copy df not currently needed:
df = df.drop(['address', 'cat1_alias', 'cat2_alias', 'cat3_alias'], axis = 1)
df.head(2)

,lat-long,name,distance,latitude,longitude,price,is_closed,rating,rating_count,my_timestamp,cat1_title,cat2_title,cat3_title
0,"43.665269,-79.319796",Maha's,1004.992087,43.67167,-79.328530,$$,False,4.0,573,2022-12-04 12:44:12.200920,Egyptian,Vegan,Sandwiches
1,"43.665269,-79.319796",The Burger's Priest,374.238073,43.66670,-79.315585,$$,False,3.5,498,2022-12-04 12:44:12.200939,Italian,NaN,NaN


flip the "is_closed" column to remove the confusing double negative, and enable foursquare comparison:

In [177]:
df['is_closed'] = df['is_closed'] == False
# Rename the heading to match:
df = df.rename(columns = {'is_closed':'is_open'})
df.head(2)

,lat-long,name,distance,latitude,longitude,price,is_open,rating,rating_count,my_timestamp,cat1_title,cat2_title,cat3_title
0,"43.665269,-79.319796",Maha's,1004.992087,43.67167,-79.328530,$$,True,4.0,573,2022-12-04 12:44:12.200920,Egyptian,Vegan,Sandwiches
1,"43.665269,-79.319796",The Burger's Priest,374.238073,43.66670,-79.315585,$$,True,3.5,498,2022-12-04 12:44:12.200939,Italian,NaN,NaN


Convert the timestamp data to the Toronto's time zone:

In [178]:
from datetime import datetime
import pytz

In [179]:
# create both timezone objects
old_timezone = pytz.timezone("US/Mountain")
new_timezone = pytz.timezone("US/Eastern")

# two-step process
localized_timestamp = old_timezone.localize(testdate)
new_testdate = localized_timestamp.astimezone(new_timezone)
new_testdate

Timestamp('2022-12-04 00:09:56.323402-0500', tz='US/Eastern')

In [180]:
# create both timezone objects
old_timezone = pytz.timezone("US/Mountain")
new_timezone = pytz.timezone("US/Eastern")

# Confirm current timezone
print(df['my_timestamp'].iloc[0])

# two-step process
for i in range(df.shape[0]):
  localized_timestamp = old_timezone.localize(df['my_timestamp'].iloc[i])
  df['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)
  df['my_timestamp'].iloc[i] = localized_timestamp.astimezone(new_timezone)


# Has converted to new timezone:
print(df['my_timestamp'].iloc[0])

2022-12-04 12:44:12.200920
2022-12-04 14:44:12.200920-05:00


In [181]:
df['distance'] = round(df['distance'], 1)
df.head(2)

,lat-long,name,distance,latitude,longitude,price,is_open,rating,rating_count,my_timestamp,cat1_title,cat2_title,cat3_title
0,"43.665269,-79.319796",Maha's,1005.0,43.67167,-79.328530,$$,True,4.0,573,2022-12-04 14:44:12.200920-05:00,Egyptian,Vegan,Sandwiches
1,"43.665269,-79.319796",The Burger's Priest,374.2,43.66670,-79.315585,$$,True,3.5,498,2022-12-04 14:44:12.200939-05:00,Italian,NaN,NaN


In [182]:
test = df['my_timestamp'].iloc[0]
print(test)
print(datetime.date(test))
print(datetime.time(test))
print(len('2022-12-04 00:09'))

2022-12-04 14:44:12.200920-05:00
2022-12-04
14:44:12.200920
16


##### Categorical encoding: price


In [183]:
print(df['price'].value_counts())
print(df['price'].isna().value_counts())
df['price'].unique()

$$      16243
$        5225
$$$      2891
$$$$      929
Name: price, dtype: int64
False    25288
True      5540
Name: price, dtype: int64


array(['$$', '$', '$$$', None, '$$$$'], dtype=object)

In [184]:
# order in array above will be same as the keys,
keys = df['price'].unique()
# Setting ordinal numerical values to match the order:
values = [2, 1, 3, None, 4]
price_map = dict(zip(keys, values))
price_map

{'$$': 2, '$': 1, '$$$': 3, None: None, '$$$$': 4}

In [185]:
# Replace
df['price'] = df['price'].map(price_map)
# Validate counts unaffected:
print(df['price'].value_counts())

df.head(1)

2.0    16243
1.0     5225
3.0     2891
4.0      929
Name: price, dtype: int64


,lat-long,name,distance,latitude,longitude,price,is_open,rating,rating_count,my_timestamp,cat1_title,cat2_title,cat3_title
0,"43.665269,-79.319796",Maha's,1005.0,43.67167,-79.32853,2.0,True,4.0,573,2022-12-04 14:44:12.200920-05:00,Egyptian,Vegan,Sandwiches


In [186]:
df['cat1_title'].value_counts()

Japanese                     6706
Thai                         3617
Italian                      3081
Breakfast & Brunch           1868
Pizza                        1866
Seafood                      1847
Egyptian                     1278
Burgers                      1264
Middle Eastern               1229
Mexican                      1187
Canadian (New)                653
Ramen                         652
Lounges                       647
Sandwiches                    646
Tapas/Small Plates            641
Vietnamese                    631
Ice Cream & Frozen Yogurt     626
American (Traditional)        613
French                        599
Aquariums                     595
Steakhouses                   582
Name: cat1_title, dtype: int64

In [187]:
df['lat-long'].value_counts().head(20)

43.665269,-79.319796             50
43.6926825,-79.441042            50
43.6617908,-79.361515            50
43.649709,-79.364184             50
43.666051,-79.374161             50
43.7005976,-79.397345            50
43.7029672,-79.3979073           50
43.68871,-79.39264               50
43.7696262,-79.5025943           50
43.76521,-79.491281              50
43.707049,-79.40966              50
43.711751,-79.378615             50
43.732975,-79.40401              50
43.660274,-79.398049             50
43.696631,-79.297436             50
43.650077,-79.391291             50
43.6765897,-79.3163022           50
43.6535,-79.354068               50
43.6704403,-79.45328549999999    50
43.6548521,-79.35397680000001    50
Name: lat-long, dtype: int64

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating